<a href="https://colab.research.google.com/github/matshch/iu5_bi_course/blob/master/lab2/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №2
## Задача №1
### Условие
Есть *N* бандитов, для каждого есть число &minus;100 < *x* < 100, которое подаётся на&nbsp;вход функции `pullBandit`:
```python
def pullBandit(bandit): #Сгенерировать случайное число
   result = np.random.randn(1) 
   if result > bandit: #Выигрыш 
        return 1 
   else: #Проигрыш 
        return -1
```
Значение результата используется в&nbsp;качестве награды.

Политика RL: policy gradient (лучше если это будет vanilla, без модификаций).

Задача: найти номер лучшего бандита.

### Решение
Импортируем все необходимые пакеты:

In [0]:
import numpy as np
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

Задаём коэффициенты бандитов:

In [0]:
bandits = [0.2, 0.1, 0.5, -5]
num_bandits = len(bandits)

Используем функцию из условия:

In [0]:
def pullBandit(bandit):
  result = np.random.randn(1)  #Сгенерировать случайное число
  if result > bandit:  #Выигрыш
    return 1
  else:  #Проигрыш
    return -1

Задаём граф обучения нейронной сети:

In [0]:
tf.reset_default_graph()

weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights, 0)

reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder, [1])
loss = -(tf.log(responsible_weight) * reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

Обучаем сеть и проверяем её решение:

In [15]:
total_episodes = 1000
total_reward = np.zeros(num_bandits)
e = 0.1  # chance of random selection

init = tf.initialize_all_variables()

with tf.Session() as sess:
  sess.run(init)
  i = 0
  while i < total_episodes:
    if np.random.rand(1) < e:
      action = np.random.randint(num_bandits)
    else:
      action = sess.run(chosen_action)

    reward = pullBandit(bandits[action])

    _, resp, ww = sess.run([update,responsible_weight,weights], feed_dict={
        reward_holder: [reward],
        action_holder: [action]})

    total_reward[action] += reward
    if i % 50 == 0:
      print(f"Running reward for the {num_bandits} bandits: {total_reward}")
    i += 1
print(f"The agent thinks bandit {np.argmax(ww)+1} is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandits)):
  print("...and it was right!")
else:
  print("...and it was wrong!")

Running reward for the 4 bandits: [1. 0. 0. 0.]
Running reward for the 4 bandits: [-1.  0.  0. 16.]
Running reward for the 4 bandits: [ 0.  2.  0. 61.]
Running reward for the 4 bandits: [  0.   3.   0. 108.]
Running reward for the 4 bandits: [  0.   2.   0. 157.]
Running reward for the 4 bandits: [ -1.   2.   0. 206.]
Running reward for the 4 bandits: [ -3.   2.   1. 253.]
Running reward for the 4 bandits: [ -5.   2.  -1. 297.]
Running reward for the 4 bandits: [ -4.   2.  -2. 345.]
Running reward for the 4 bandits: [ -5.   4.  -2. 392.]
Running reward for the 4 bandits: [ -6.   5.  -4. 438.]
Running reward for the 4 bandits: [ -6.   7.  -5. 481.]
Running reward for the 4 bandits: [ -8.   7.  -7. 527.]
Running reward for the 4 bandits: [ -8.   9.  -6. 574.]
Running reward for the 4 bandits: [ -8.   9.  -5. 623.]
Running reward for the 4 bandits: [-10.   8.  -6. 667.]
Running reward for the 4 bandits: [-10.   6.  -7. 714.]
Running reward for the 4 bandits: [-10.   6.  -7. 764.]
Running 